In [1]:
#from tensorflow.compat.v1 import ConfigProto
#from tensorflow.compat.v1 import InteractiveSession

#config = ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

# from tensorflow.keras.applications.ResNet152V2
#from tensorflow.keras.applications import Xception
# from keras.applications.resnet_152_v2 import preprocess_input
#from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]



train_path = "C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train"
valid_path = "C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\val"


In [4]:
#EDA
import os
os.listdir("C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train")

len(os.listdir("C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train"))

10

In [5]:


vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [6]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
  # useful for getting number of output classes
folders = glob('C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train\\*')

In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [9]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [10]:

# view the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [12]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:

training_set = train_datagen.flow_from_directory('C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


Found 10000 images belonging to 10 classes.


In [14]:
test_set = test_datagen.flow_from_directory('C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1000 images belonging to 10 classes.


In [15]:

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\praja\AppData\Local\Temp\ipykernel_10916\2245298814.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/20
313/313 [==============================] - 1635s 5s/step - loss: 0.9030 - accuracy: 0.7031 - val_loss: 0.8833 - val_accuracy: 0.7320
Epoch 2/20
313/313 [==============================] - 1828s 6s/step - loss: 0.4405 - accuracy: 0.8480 - val_loss: 0.6187 - val_accuracy: 0.8090
Epoch 3/20
313/313 [==============================] - 1812s 6s/step - loss: 0.3840 - accuracy: 0.8700 - val_loss: 0.7690 - val_accuracy: 0.7790
Epoch 4/20
313/313 [==============================] - 1771s 6s/step - loss: 0.3268 - accuracy: 0.8907 - val_loss: 0.5479 - val_accuracy: 0.8390
Epoch 5/20
313/313 [==============================] - 1782s 6s/step - loss: 0.2858 - accuracy: 0.9001 - val_loss: 0.6546 - val_accuracy: 0.8220
Epoch 6/20
313/313 [==============================] - 1828s 6s/step - loss: 0.2653 - accuracy: 0.9085 - val_loss: 0.7303 - val_accuracy: 0.8100
Epoch 7/20
313/313 [==============================] - 1825s 6s/step - loss: 0.2105 - accuracy: 0.9280 - val_loss: 0.6563 - val_accuracy:

In [16]:
import pickle
pickle.dump(model, open('model_vgg16.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://782e9ac2-8fa8-461d-a35c-8dc268aa912c/assets


INFO:tensorflow:Assets written to: ram://782e9ac2-8fa8-461d-a35c-8dc268aa912c/assets


In [17]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')

In [18]:
y_pred = model.predict(test_set)

32/32 [==============================] - 114s 4s/step


In [19]:
from keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model("C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\MainProjectInceptionV3\\model_vgg16.h5")


In [20]:
print(train_path)

C:\Users\praja\OneDrive\Documents\Abhi\tomato\train


In [21]:
#list of classes with their Indices
training_set.class_indices


{'Tomato___Bacterial_spot': 0,
 'Tomato___Early_blight': 1,
 'Tomato___Late_blight': 2,
 'Tomato___Leaf_Mold': 3,
 'Tomato___Septoria_leaf_spot': 4,
 'Tomato___Spider_mites Two-spotted_spider_mite': 5,
 'Tomato___Target_Spot': 6,
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7,
 'Tomato___Tomato_mosaic_virus': 8,
 'Tomato___healthy': 9}

In [22]:
#list of classes in Dataset (array format)
training_set.class_indices.keys()

dict_keys(['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy'])

In [23]:
list(training_set.class_indices.keys())

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

In [24]:
#make dictionary of all disease
ref=dict(zip(list(training_set.class_indices.values()) ,list(training_set.class_indices.keys())))


In [33]:
import numpy as np
from keras.applications.inception_v3 import preprocess_input


def prediction(path):
   
    img=image.load_img(path, target_size=(224,224))
    
    i=image.img_to_array(img)
#     i.shape
    im=preprocess_input(i)
    
    img=np.expand_dims(im, axis=0)
    
    pred=np.argmax(model.predict(img))
    
    print(f" the image belongs to {ref[pred]}")

In [34]:
path="C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\tomato\\train\\Tomato___healthy\\84cbb98b-5c2f-4367-8d78-52be491e66bd___GH_HL Leaf 336.JPG"
prediction(path)

1/1 [==============================] - 0s 140ms/step
 the image belongs to Tomato___healthy


In [36]:
Path="C:\\Users\\praja\\OneDrive\\Documents\\Abhi\\MainProjectInceptionV3\\tomato\\val\\Tomato___healthy\\0e1d7722-3df9-4a3a-a4e2-bc0ea3ce0da4___RS_HL 9979.JPG"
prediction(path)

1/1 [==============================] - 0s 132ms/step
 the image belongs to Tomato___healthy
